# Performance Metrics

This notebook demonstrates how yippy computes coronagraph performance metrics
and, critically, **how those metrics are consumed by exposure time calculators**
in downstream science codes.

**Reference**: [Stark et al. (2025)](https://arxiv.org/abs/2502.18556) —
*Cross-Model Validation of Coronagraphic Exposure Time Calculators for the
Habitable Worlds Observatory*

## Metrics Overview

| Metric | yippy accessor | Description | Units |
|--------|---------------|-------------|-------|
| **Core Throughput** | `coro.throughput(r)` | Fraction of planet flux inside photometric aperture | dimensionless |
| **Raw Contrast** | `coro.raw_contrast(r)` | Stellar leakage relative to planet signal | dimensionless |
| **Occulter Transmission** | `coro.occulter_transmission(r)` | Sky transmission mask radial profile | dimensionless |
| **Core Area** | `coro.core_area(r)` | Effective solid angle of the PSF core | (λ/D)² |
| **Core Mean Intensity** | `coro.core_mean_intensity(r)` | Mean stellar intensity per core pixel | dimensionless |
| **Noise Floor** | `coro.noise_floor(r)` | Systematic noise floor from stellar PSF subtraction residuals | dimensionless |

---
## Setup

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from yippy.datasets import fetch_coronagraph
from yippy import Coronagraph

# Download (or use cached) example coronagraph
yip_path = fetch_coronagraph()
print(f"YIP path: {yip_path}")

# Load coronagraph — performance curves are computed automatically
coro = Coronagraph(yip_path)
print(f"\nCoronagraph: {coro.name}")
print(f"Off-axis type: {coro.offax.type}")
print(f"Number of PSFs: {coro.offax.n_psfs}")
print(f"Pixel scale: {coro.pixel_scale}")
print(f"IWA: {coro.IWA:.2f}")
print(f"OWA: {coro.OWA:.2f}")

## 1. Core Throughput ($\Upsilon_c$)

Throughput measures how much of a point source's flux is preserved by the coronagraph
at each angular separation. It is the fraction of the off-axis PSF flux that falls
within a photometric aperture:

$$\Upsilon_c(r) = \frac{\text{flux in aperture at separation } r}{\text{total PSF flux}}$$

**In the ETC**, throughput directly scales the planet count rate $C_p$. It is the
single most important metric for planet detectability.

In [ ]:
from yippy.performance import compute_throughput_curve

sep, throughput = compute_throughput_curve(coro, aperture_radius_lod=0.7)

fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(sep, throughput, 'o-', ms=5, color='#4CAF50')
ax.axvline(coro.IWA.value, ls='--', color='gray', alpha=0.7, label=f'IWA = {coro.IWA.value:.1f} λ/D')
ax.set_xlabel('Separation [λ/D]')
ax.set_ylabel('Throughput (Υ)')
ax.set_title(f'{coro.name} — Core Throughput')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Number of PSF positions: {len(sep)}")
print(f"Throughput range: [{throughput.min():.4f}, {throughput.max():.4f}]")

## 2. Raw Contrast ($C$)

Raw contrast measures the residual stellar leakage at each separation, relative to
the off-axis planet signal:

$$C(r) = \frac{F_{\star,\text{leaked}}(r)}{F_{\text{planet}}(r)}$$

where both fluxes are measured in the same photometric aperture.

**In the ETC (EXOSIMS fallback path)**: When `core_mean_intensity` is not available,
EXOSIMS computes stellar leakage as $C_{sr} = C_\star \cdot C(r) \cdot \Upsilon_c(r)$.

**In the ETC (noise floor)**: AYO's noise floor is conceptually related to
contrast — it bounds the achievable contrast after PSF subtraction.

In [ ]:
from yippy.performance import compute_raw_contrast_curve

sep_c, contrast = compute_raw_contrast_curve(coro, aperture_radius_lod=0.7)

fig, ax = plt.subplots(figsize=(8, 5))
ax.semilogy(sep_c, contrast, 'o-', ms=5, color='#E91E63')
ax.axvline(coro.IWA.value, ls='--', color='gray', alpha=0.7, label=f'IWA = {coro.IWA.value:.1f} λ/D')
ax.set_xlabel('Separation [λ/D]')
ax.set_ylabel('Raw Contrast')
ax.set_title(f'{coro.name} — Raw Contrast Curve')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Contrast range: [{contrast.min():.2e}, {contrast.max():.2e}]")

## 3. Occulter Transmission ($T_{sky}$ / `occ_trans`)

The occulter transmission represents how much light from spatially extended
background sources (zodiacal dust, exozodiacal dust) passes through the
coronagraph mask. It is the radial profile of the `sky_trans.fits` mask.

**In the ETC**, it scales all extended-source backgrounds. AYO approximates
the coronagraph's effect on extended sources by convolving the spatially-dependent
PSF with a uniform background, producing this throughput factor.

In [ ]:
from yippy.performance import compute_occ_trans_curve

sep_ot, occ_trans = compute_occ_trans_curve(coro)

fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(sep_ot, occ_trans, '-', color='#FF9800', lw=2)
ax.axvline(coro.IWA.value, ls='--', color='gray', alpha=0.7, label=f'IWA = {coro.IWA.value:.1f} λ/D')
ax.set_xlabel('Separation [λ/D]')
ax.set_ylabel('Occulter Transmission (Θ)')
ax.set_title(f'{coro.name} — Occulter Transmission')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Core Area ($\Omega$)

Core area is the effective solid angle of the PSF core at each separation,
in units of $(\lambda/D)^2$. It can be computed two ways:

- **Fixed aperture** (default): $\Omega = \pi r_{ap}^2$ — constant at all separations
- **Gaussian fit**: Fit a 2D Gaussian to each PSF and compute $\Omega = \pi \cdot \text{FWHM}_x \cdot \text{FWHM}_y / 4$

**In the ETC**, core area appears in:
- Every background term: $C_{b,\star}$, $C_{bz}$, $C_{bez}$, $C_{bth}$ are all proportional to $\Omega$
- Detector noise: $N_{\rm pix} = \Omega / \theta_{\rm det}^2$
- AYO optimizes over a set of `psf_trunc_ratio` values, each giving a different $\Omega$

In [ ]:
from yippy.performance import compute_core_area_curve

# Fixed aperture
sep_a, area_fixed = compute_core_area_curve(coro, aperture_radius_lod=0.7, fit_gaussian=False)

# Gaussian fit
sep_g, area_gauss = compute_core_area_curve(coro, aperture_radius_lod=0.7, fit_gaussian=True)

fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(sep_a, area_fixed, 's-', ms=5, color='#9C27B0', label='Fixed aperture')
ax.plot(sep_g, area_gauss, 'o-', ms=5, color='#2196F3', label='Gaussian fit')
ax.set_xlabel('Separation [λ/D]')
ax.set_ylabel('Core Area Ω [(λ/D)²]')
ax.set_title(f'{coro.name} — Core Area')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Core Mean Intensity ($\bar{I}_\star$)

Core mean intensity is the average stellar leakage intensity in the PSF core
region as a function of separation. It is computed for each available stellar
angular diameter from the `stellar_intens.fits` data.

**In the AYO path**: The raw `Istar[x,y]` 2D map is used per-pixel. The map
is a 3D cube `[npix, npix, ndiams]` interpolated to the target's angular diameter.

**In the EXOSIMS path**: `core_mean_intensity(λ, WA, d★)` is a 1D radial
function (with stellar diameter as a third argument). The total leaked
starlight in the core is:
$$C_{sr} = C_\star \cdot \bar{I}_\star \cdot \frac{\Omega}{p_{\text{core}}^2}$$

In [ ]:
from yippy.performance import compute_core_mean_intensity_curve

sep_ci, intensities = compute_core_mean_intensity_curve(coro)

fig, ax = plt.subplots(figsize=(8, 5))
for diam, profile in intensities.items():
    ax.semilogy(sep_ci, profile, '-', label=f'Diam = {diam.value:.1f} λ/D')
ax.axvline(coro.IWA.value, ls='--', color='gray', alpha=0.7, label=f'IWA = {coro.IWA.value:.1f} λ/D')
ax.set_xlabel('Separation [λ/D]')
ax.set_ylabel('Core Mean Intensity')
ax.set_title(f'{coro.name} — Core Mean Intensity')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. All Curves — Summary Panel

The `Coronagraph` class pre-computes all performance curves during initialization
and stores spline interpolators for fast evaluation at arbitrary separations.

In [ ]:
# Evaluate at arbitrary separations using pre-computed interpolators
seps = np.linspace(coro.IWA.value, coro.OWA.value, 200)

fig, axes = plt.subplots(2, 3, figsize=(15, 9))

# Throughput
ax = axes[0, 0]
ax.plot(seps, coro.throughput(seps), color='#4CAF50', lw=2)
ax.set_ylabel('Throughput (Υ)')
ax.set_title('Core Throughput\n→ scales $C_p$')
ax.grid(True, alpha=0.3)

# Raw Contrast
ax = axes[0, 1]
ax.semilogy(seps, coro.raw_contrast(seps), color='#E91E63', lw=2)
ax.set_ylabel('Raw Contrast')
ax.set_title('Raw Contrast\n→ EXOSIMS $C_{sr}$ (fallback)')
ax.grid(True, alpha=0.3)

# Occulter Transmission
ax = axes[0, 2]
ax.plot(seps, coro.occulter_transmission(seps), color='#FF9800', lw=2)
ax.set_ylabel('Occulter Transmission (Θ)')
ax.set_title('Occulter Transmission\n→ scales $C_{bz}$, $C_{bez}$')
ax.grid(True, alpha=0.3)

# Core Area
ax = axes[1, 0]
ax.plot(seps, coro.core_area(seps), color='#9C27B0', lw=2)
ax.set_ylabel('Core Area Ω [(λ/D)²]')
ax.set_title('Core Area\n→ all backgrounds, $N_{pix}$')
ax.grid(True, alpha=0.3)

# Core Mean Intensity
ax = axes[1, 1]
ax.semilogy(seps, coro.core_mean_intensity(seps), color='#00BCD4', lw=2)
ax.set_ylabel('Core Mean Intensity')
ax.set_title('Core Mean Intensity\n→ $C_{b★}$ / $C_{sr}$')
ax.grid(True, alpha=0.3)

# Noise Floor
ax = axes[1, 2]
ax.semilogy(seps, coro.noise_floor(seps), color='#795548', lw=2)
ax.set_ylabel('Noise Floor')
ax.set_title('Noise Floor\n→ $C_{nf}$ (AYO), $C_{sp}$ via ppFact (EXOSIMS)')
ax.grid(True, alpha=0.3)

for ax in axes.flat:
    ax.set_xlabel('Separation [λ/D]')
    ax.axvline(coro.IWA.value, ls='--', color='gray', alpha=0.5)

fig.suptitle(f'{coro.name} — Performance Summary', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 7. Off-Axis PSF Format and x-Axis Selection

Performance metrics are computed along the **x-axis** ($y = 0$) of the off-axis PSF grid.
This works for all coronagraph types:

- **1D** (radially symmetric): offsets along x only — all positions are on the x-axis
- **2Dq** (quarter symmetric): offsets in the first quadrant — code selects `y_idx = argmin(|y_offsets|)`
- **2Df** (full 2D): offsets in all quadrants — same y≈0 selection

In [ ]:
import astropy.units as u

print(f"Coronagraph type: {coro.offax.type}")
print(f"  1d  = radially symmetric (offsets along x only)")
print(f"  2dq = 2D quarter symmetric (first quadrant)")
print(f"  2df = 2D full (all quadrants)")
print()
print(f"x_offsets ({len(coro.offax.x_offsets)}): {coro.offax.x_offsets}")
print(f"y_offsets ({len(coro.offax.y_offsets)}): {coro.offax.y_offsets}")

# Show which y_offset is selected for performance metrics
y_offsets = np.array(coro.offax.y_offsets)
y_idx = int(np.argmin(np.abs(y_offsets)))
print(f"\nPerformance curve y-offset: y_offsets[{y_idx}] = {y_offsets[y_idx]:.4f} λ/D")
print(f"This {'is' if y_offsets[y_idx] == 0 else 'is NOT'} exactly y=0")

## 8. Aperture Radius Effect

The photometric aperture radius affects throughput, contrast, and core area.
AYO optimizes over multiple `psf_trunc_ratio` values to minimize integration
time at each separation. The standard benchmark uses 0.7 λ/D.

In [ ]:
radii = [0.5, 0.7, 0.85, 1.0]
colors = ['#E91E63', '#4CAF50', '#2196F3', '#FF9800']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

for r, c in zip(radii, colors):
    s, t = compute_throughput_curve(coro, aperture_radius_lod=r)
    ax1.plot(s, t, 'o-', ms=4, color=c, label=f'r = {r} λ/D')
    
    s, con = compute_raw_contrast_curve(coro, aperture_radius_lod=r)
    ax2.semilogy(s, con, 'o-', ms=4, color=c, label=f'r = {r} λ/D')

ax1.set_xlabel('Separation [λ/D]')
ax1.set_ylabel('Throughput')
ax1.set_title('Throughput vs Aperture Radius')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.set_xlabel('Separation [λ/D]')
ax2.set_ylabel('Raw Contrast')
ax2.set_title('Raw Contrast vs Aperture Radius')
ax2.legend()
ax2.grid(True, alpha=0.3)

fig.suptitle(f'{coro.name} — Aperture Radius Comparison', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---
## How Metrics Feed Into Three Exposure Time Calculators

Three codes consume yippy's metrics to calculate integration times:
**AYO** (C), **pyEDITH** (Python, based on AYO), and **EXOSIMS** (Python).
Each uses a slightly different formulation.

### The AYO / pyEDITH Exposure Time Equation

AYO (Stark et al. 2014, 2015, 2019, 2024) computes:

$$t = \text{SNR}^2 \cdot \frac{C_p + 2\,C_b}{C_p^2 - C_{nf}^2}$$

The **factor of 2** on all background count rates comes from the
assumption of ADI (Angular Differential Imaging) PSF subtraction:
the background noise doubles because it appears in both the science
and reference images. This is implemented in the C code as:
```c
twoCRb = CRb_multiplier * CRb;  // CRb_multiplier = 2
cp = (CRp + twoCRb) / (CRp*CRp - SNRCRpfloor*SNRCRpfloor);
t = SNR^2 * cp;
```

The **noise floor** $C_{nf}$ represents a systematic limit that
cannot be reduced with more integration time. When $C_p < C_{nf}$,
exposure time is infinite — the planet is undetectable.

**pyEDITH** extends this with an exozodi noise floor:
$$C_{nf} = \sqrt{C_{nf,\star}^2 + C_{nf,ez}^2}$$
where $C_{nf,ez} = \text{SNR} \cdot C_{bez} / \text{PPF}_{ez}$ accounts
for imperfect exozodiacal light subtraction.

### The EXOSIMS Exposure Time Equation

EXOSIMS uses a different formulation (derived from SNR = $C_p\sqrt{t} / \sqrt{C_b t + (C_{sp} t)^2}$):

$$t = \text{SNR}^2 \cdot \frac{C_b}{C_p^2 - (\text{SNR} \cdot C_{sp})^2}$$

Key differences from AYO:
- **No factor of 2** on backgrounds — PSF subtraction effects are
  encoded differently
- The systematic limit is a **speckle residual** $C_{sp} = C_{sr} \cdot \text{ppFact} \cdot \text{stabilityFact}$
  rather than a separate noise floor
- `ppFact` (post-processing factor) plays the role of AYO's noise floor

### Key Architectural Differences

| | AYO / pyEDITH | EXOSIMS |
|---|---|---|
| **Data format** | Full 2D maps — per-pixel lookup at planet position | 1D radial curves — function of (λ, WA) |
| **Stellar leakage** | `Istar[x,y] / pixscale² × omega` | `core_mean_intensity × Ω/platescale²` or `core_contrast × core_thruput` |
| **Noise floor** | Stddev of differenced Istar maps × Ω | `C_sr × ppFact × stabilityFact` |
| **PSF subtraction** | Factor of 2 on all backgrounds (ADI) | No factor of 2; PSF subtraction in ppFact |
| **Aperture optimization** | Loops over `psf_trunc_ratio` array | Single fixed aperture |
| **Stellar diameter** | Interpolates 3D Istar cube [x,y,diam] | `core_mean_intensity(λ, WA, d★)` callable |

### Count Rate Equations

All three codes compute the same physical count rates, using slightly
different representations:

#### Planet signal ($C_p$)

| Code | Formula |
|------|--------|
| AYO/pyEDITH | $C_p = F_0 \cdot 10^{-0.4 m_\star} \cdot 10^{-0.4 \Delta m_p} \cdot A \cdot T \cdot \Delta\lambda \cdot N_{ch} \cdot \Upsilon_c(x,y)$ |
| EXOSIMS | $C_p = C_\star \cdot 10^{-0.4 \Delta m} \cdot \text{core\_thruput}(\lambda, \text{WA})$ |

→ Uses **core throughput** ($\Upsilon_c$ / `core_thruput`)

#### Stellar leakage ($C_{b,\star}$ / $C_{sr}$)

| Code | Formula |
|------|--------|
| AYO/pyEDITH | $C_{b,\star} = F_\star \cdot \frac{I_\star(x,y)}{\theta^2} \cdot \Omega(x,y) \cdot A \cdot T \cdot \Delta\lambda \cdot N_{ch}$ |
| EXOSIMS (preferred) | $C_{sr} = C_\star \cdot \overline{I}_\star(\lambda, \text{WA}, d_\star) \cdot \frac{\Omega(\lambda,\text{WA})}{p_{\text{core}}^2}$ |
| EXOSIMS (fallback) | $C_{sr} = C_\star \cdot C_{\text{contrast}}(\lambda,\text{WA}) \cdot \Upsilon_c(\lambda,\text{WA})$ |

→ Uses **stellar intensity** (`Istar` / `core_mean_intensity`) or **raw contrast** × **throughput**

#### Zodiacal light ($C_{b,zodi}$ / $C_z$)

| Code | Formula |
|------|--------|
| AYO/pyEDITH | $C_{b,zodi} = F_{zodi} \cdot (\lambda/D)_{\rm arcsec}^2 \cdot T_{sky}(x,y) \cdot \Omega(x,y) \cdot A \cdot T \cdot \Delta\lambda \cdot N_{ch}$ |
| EXOSIMS | $C_z = F_0 \cdot \text{losses} \cdot f_Z \cdot \Omega(\lambda,\text{WA}) \cdot \text{occ\_trans}(\lambda,\text{WA})$ |

→ Uses **occulter transmission** (`skytrans` / `occ_trans`) × **core area** ($\Omega$ / `core_area`)

#### Exozodiacal light ($C_{b,exo}$ / $C_{ez}$)

Same structure as zodiacal light but with $1/(d \cdot s)^2$ distance scaling.
EXOSIMS can optionally use `core_thruput` instead of `occ_trans` for the
exozodi term (`use_core_thruput_for_ez` flag).

#### Thermal background ($C_{b,th}$)

$$C_{b,th} = B_\lambda(T_{\rm tel}) \cdot \Delta\lambda \cdot A \cdot (\lambda/D)_{\rm rad}^2 \cdot \epsilon \cdot \text{QE} \cdot \Omega$$

→ Uses **core area** ($\Omega$)

#### Detector noise ($C_{b,det}$)

$$C_{b,det} = N_{\rm pix} \cdot \left(\xi + \frac{\text{RN}^2}{\tau_{\rm read}} + \frac{\text{CIC}}{t_{\rm photon}}\right)$$

where $N_{\rm pix} = \Omega / \theta_{\rm det}^2$ → Uses **core area** ($\Omega$)

#### Noise floor ($C_{nf}$) — AYO/pyEDITH only

$$C_{nf} = \text{SNR} \cdot F_\star \cdot \frac{\sigma_{nf}(x,y)}{\theta^2} \cdot \Omega(x,y) \cdot A \cdot T \cdot \Delta\lambda \cdot N_{ch}$$

The noise floor map $\sigma_{nf}$ is computed from the stddev of
photometric-aperture-integrated differences between two stellar
intensity maps ("roll 1" and "roll 2").

→ Uses **noise floor** map (`noisefloor`)

### Summary: Metric → ETC Variable Mapping

| yippy metric | AYO variable | EXOSIMS variable | pyEDITH variable | Used by |
|---|---|---|---|---|
| `throughput` | `photap_frac[x,y,ratio]` | `core_thruput(λ, WA)` | `Υ` | $C_p$ |
| `raw_contrast` | `Istar/PSFpeak` | `core_contrast(λ, WA)` | — | $C_{sr}$ (fallback) |
| `occulter_transmission` | `skytrans[x,y]` | `occ_trans(λ, WA)` | `skytrans` | $C_{bz}$, $C_{bez}$ |
| `core_area` | `omega_lod[x,y,ratio]` | `core_area(λ, WA)` | `omega_lod` | All backgrounds, $N_{pix}$ |
| `core_mean_intensity` | `Istar[x,y]` | `core_mean_intensity(λ, WA, d★)` | `Istar` | $C_{b\star}$ / $C_{sr}$ |
| `noise_floor` | `noisefloor[x,y]` | — (uses `ppFact` instead) | `noisefloor` | $C_{nf}$ (AYO), $C_{sp}$ (EXOSIMS) |